# Building lattices from unitcells

For most lattice based calculations, a `Lattice` object is needed that has to be constructed at some point to deliver the lattice connectivity information. Usually, the preceiding step is to obtain the lattice geometry in form of a `Unitcell` object, which is usually obtained from a pre-implemented unitcell. Details on how to obtain the unitcell can be found in [this tutorial](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/unitcells/pre-implemented_unitcells.ipynb).

The following tutorial aims on providing an understanding how to transform the periodic unitcell into a finitely extended lattice ob a desired shape and periodicity. For this purpose exists a subpackage of `LatticePhysics`, called `LatPhysLatticeConstruction` in which all lattice constructing routines are collected.

In [ ]:
using LatPhysBase
using LatPhysUnitcellLibrary
using LatPhysLatticeConstruction

### Reminder: Obtaining unitcells

As a brief reminder and a basis for the following tutorial, let us get two unitcells from the pre-implemented collection - a 2d and a 3d unitcell. These unitcells are used in the following to construct lattices.

In [ ]:
# 2D unitcell is honeycomb
unitcell_2d = getUnitcellHoneycomb(4);

# 3D unitcell is pyrochlore
unitcell_3d = getUnitcellPyrochlore(1);

### Overview over lattice construction routines

The module `LatPhysLatticeConstruction` has various pre-implemented routines for building lattices. These include
- lattices from regular patterns with different boundary conditions
- lattices built by bond distance from an origin site
- lattices built within a given shape in real space

The following sections will explain the relevant functions in greater detail.


### Periodic lattices

The most abundant shape of lattices is a regular arrangement of unitcells with periodic boundary conditions. These lattices are characterized by their linear extent in different Bravais lattice directions. To construct such lattices, one can use the function
```
function getLatticePeriodic(
        unitcell :: U,
        extent   :: NTuple{N,Int64}
    ) :: Lattice{S,B,U} where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B}}
```
as seen in the following examples

In [ ]:
# construct a 2D lattice which is 5x10 unitcells in size
getLatticePeriodic(unitcell_2d, (5,10))

In [ ]:
# construct a 3D lattice which is 5x2x3 unitcells in size
getLatticePeriodic(unitcell_3d, (5,2,3))

##### Periodic lattices of equal linear extent L

Since in many cases one is interested in a lattice of size LxLxL..., there is a short hand notation introduced by
```
function getLatticePeriodic(
        unitcell :: U,
        extent   :: Int64
    ) :: Lattice{S,B,U} where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B}}
```
which simply calls the aforementioned function respectively.

In [ ]:
# construct a 2D lattice which is 7x7 unitcells in size
getLatticePeriodic(unitcell_2d, 7)

In [ ]:
# construct a 3D lattice which is 4x4x4 unitcells in size
getLatticePeriodic(unitcell_3d, 4)

##### Periodic lattices for lattices with an explicit type signature

Similar to how pre-implemented unitcells have a dispatch tree in the background, all lattice construction related functions have a similar dispatch tree as well. In general, there is a common syntax for function names which reroute among themselves and encapsulate information.

Most of the core functions are on the footing of abstract types. In this case, there is a core function `getLatticePeriodic` which needs not only the unitcell and extent information, but also the lattice type (subtype of `AbstractLattice`) to determine which lattice type to build. The function syntax is
```
function getLatticePeriodic(
        :: Type{L},
        unitcell :: U,
        extent   :: Int64
    ) :: L where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B},SL,BL,L<:AbstractLattice{SL,BL,U}}
```
The aforementioned functions are solely wrappers around this core function and feed it the default lattice implementation `Lattice` in the desired type structure. 

If one wants to use this explicity type signature function nevertheless (e.g. in case one has a custom lattice type), than a suitable example would look like

In [ ]:
# construct a 10x5 periodic 2d lattice with explicit type
lt = getLatticePeriodic(
    Lattice{Site{Int64,2}, Bond{Int64,2}, Unitcell{Site{Int64,2},Bond{Int64,2}}},
    unitcell_2d,
    (10,5)
)

### Lattices built by bond distance

In some cases one is interested in a finite cluster of sites that span by bond distance around an origin site. These lattice building functions are summarized under the syntax `getLatticeByBondDistance`.

The function syntax is the following:
```
function getLatticeByBondDistance(
        unitcell     :: U,
        bonddistance :: Integer,
        origin       :: Integer = 1
    ) :: Lattice{S,Bond{LB,0},U} where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B}}
```

Note that one only needs a unitcell, the desired bond distance up to which the lattice is constructed, as well as the starting site within the unitcell (as an optional parameter).

Examples are

In [ ]:
# construct a lattice up to bond distance 15 in 2D
lt = getLatticeByBondDistance(unitcell_2d, 15)

In [ ]:
# construct a lattice up to bond distance 15 in 2D originating from site 2 of the unitcell
lt = getLatticeByBondDistance(unitcell_2d, 15, 2)

In [ ]:
# construct a lattice up to bond distance 10 in 3D
lt = getLatticeByBondDistance(unitcell_3d, 10)

##### Lattices by bond distance with an explicit type signature

Similar to periodic lattices, lattice building by bond distance also comes with a general interface in which a lattice type (subtype of `AbstractLattice`) can be used explicitly. The function syntax is
```
function getLatticeByBondDistance
        :: Type{L},
        unitcell     :: U,
        bonddistance :: Integer,
        origin       :: Integer = 1
    ) :: L where {N,LB,B<:AbstractBond{LB,0},S,U<:AbstractUnitcell{S,B},SL,BL,L<:AbstractLattice{SL,BL,U}}
```
The aforementioned functions are solely wrappers around this core function and feed it the default lattice implementation `Lattice` in the desired type structure. Again, if one wants to use this explicity type signature function nevertheless (e.g. in case one has a custom lattice type), than a suitable example would look like

In [ ]:
# construct a lattice up to bond distance 8 in 2D with explicit lattice type
lt = getLatticeByBondDistance(
    Lattice{Site{Int64,2}, Bond{Int64,0}, Unitcell{Site{Int64,2},Bond{Int64,2}}},
    unitcell_2d,
    8
)

### Lattices built inside a given shape

Mostly for plotting purposes, there is yet another convinient way to generate a lattice - building it inside a given shape, spreading outward from an origin site. This way of generating lattices results in a lattice which is a wanted shape (e.g. a rectangle or box) and which can be plotted very nicely. However, the lattice again has open boundary conditions, i.e. no periodic directions.

Building inside a given very general shape can be accomplished with the function
```
function getLatticeInShape(
        unitcell :: U,
        shape    :: Function,
        origin   :: Integer = 1
    ) :: Lattice{S,Bond{LB,0},U} where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B}}

```
where `shape` is a function that returns a `Bool` for any input coordinate vector to determine if this coordinate is still inside the desired shape. To illustrate, look at the example (in 2d) below that constructs a lattice within a sphere

In [ ]:
# shape function of the sphere in 2D: is the point inside the circle with radius 3.0 or not
shape(position :: Vector{Float64}) = sum(position.*position) < 3.0*3.0

# build the lattice inside this shape
lt = getLatticeInShape(unitcell_2d, shape)

##### Special shape 1 - sphere

A very common special shape is a sphere. For building a lattice inside a sphere, there exists a wrapper function around the general shape building function with the syntax
```
function getLatticeInSphere(
        unitcell   :: U,
        radius     :: Real,
        center     :: Vector{<:Real},
        origin     :: Integer = 1
    ) :: Lattice{S,Bond{LB,0},U} where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B}}
```
where one simply has to pass the radius and center of the sphere respectively. Note that `origin` refers to the building origin of the lattice whereas `center` determines the shapes center.

In [ ]:
# build a 2d lattice inside a circle of radius 3 around [0,0]
lt = getLatticeInSphere(unitcell_2d, 3, [0,0])

In [ ]:
# build a 3d lattice inside a sphere of radius 3 around [0,0,0]
lt = getLatticeInSphere(unitcell_3d, 3, [0,0,0])

##### Special shape 2 - box

Another very common shape (especially for plotting) is the shape of a box (or rectangle in 2d). There is a pre-implemented function that generates lattices inside a box with the syntax
```
function getLatticeInBox(
        unitcell   :: U,
        dimensions :: Vector{<:Real},
        center     :: Vector{<:Real},
        origin     :: Integer = 1
    ) :: Lattice{S,Bond{LB,0},U} where {N,LB,B<:AbstractBond{LB,N},S,U<:AbstractUnitcell{S,B}}
```
where `dimensions` refers to the extent in real space units along principal real space axis (and *not* unitcell Bravais lattice vectors). Further, note again that `origin` refers to the building origin of the lattice whereas `center` determines the shapes center.

In [ ]:
# build a 2d lattice in a box of dimension 10x7 around [0,0]
lt = getLatticeInBox(unitcell_2d, [10,7], [0,0])

In [ ]:
# build a 3d lattice in a box of dimension 5x3x7 around [0,0,0]
lt = getLatticeInBox(unitcell_2d, [5,3,7], [0,0,0])